In [1]:
import boto3,os
import sagemaker as sage
from sagemaker import get_execution_role
from time import gmtime,strftime

In [2]:
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'sagemaker-nmf-container'
tag = ':nmflatest'
training_repository_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{ecr_repository}'

role = get_execution_role()
sess = sage.Session()

In [17]:
!docker build -t $ecr_repository .

Sending build context to Docker daemon   42.5kB
Step 1/10 : FROM python:3.7-slim-buster
 ---> e7d86653f62f
Step 2/10 : MAINTAINER ASC "mbod@asc.upenn.edu jmparelman@gmail.com"
 ---> Using cache
 ---> a2426e1e7b5b
Step 3/10 : RUN mkdir -p /data/input /data/output
 ---> Using cache
 ---> e89d28149bc8
Step 4/10 : RUN apt-get -y update && apt-get install -y --no-install-recommends         build-essential         python-dev         python3-pip     && pip3 install -U pip
 ---> Using cache
 ---> a95a37ea34c1
Step 5/10 : RUN     pip3 install -U pandas          joblib          sklearn
 ---> Using cache
 ---> a76a301f7eef
Step 6/10 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 2cd5171a4e08
Step 7/10 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 99bc4a23967f
Step 8/10 : ENV PATH="/opt/ml:${PATH}"
 ---> Using cache
 ---> 97ccddcefd72
Step 9/10 : COPY scripts/run_nmf.py /opt/ml/run_nmf.py
 ---> d5dd5b91a71c
Step 10/10 : ENTRYPOINT ["python3","/opt/ml/run_nmf.py"]
 ---> Running in

In [18]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 064258348567.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [19]:
!aws ecr create-repository --repository-name $ecr_repository


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sagemaker-nmf-container' already exists in the registry with id '064258348567'


In [20]:
!docker tag {ecr_repository} "064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container:latest"
!docker push "064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container:latest"

The push refers to repository [064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container]

2834dac4: Preparing 
4d9b5eab: Preparing 
afb0d5e9: Preparing 
c7543451: Preparing 
3c6439bc: Preparing 
6bb9ffea: Preparing 
663e622b: Preparing 
845af46d: Preparing 
2834dac4: Pushed lready exists 2kB2Klatest: digest: sha256:762b9c2d5af80f0e92e595913ef0711e8c5dda541f7ca4fd8294d4f51282d10c size: 2211


In [6]:
nmf = sage.estimator.Estimator( "064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container:latest",
                              role,1, 'ml.m4.16xlarge',
                              output_path = 's3://ascsagemaker/JMP_congressional_nmf/models',
                              sagemaker_session=sess,
                              max_run=9000)

# START JOBS

In [4]:
congress_list = range(111,115)
for chamber in congress_list:
    cstr = '{:0>3}'.format(chamber)
    print(cstr)

111
112
113
114


In [7]:
for chamber in congress_list:
    cstr = '{:0>3}'.format(chamber)
    print('Scheduling', chamber)
    nmf.fit(f's3://ascsagemaker/JMP_congressional_nmf/latest_dtms/{cstr}.pkl',wait=False)

Scheduling 111
Scheduling 112
Scheduling 113
Scheduling 114
